# Time-Series Features

This notebook focuses on extracting an```nalyzing time-series specific features from stock```rket data for```e StockPredictionPro platform.

**Time-Series Feature Types:**
- Lag features (price and volume lags)
- Rolling statistics (mean, standard deviation, min, max)
- Exponentially weighted metrics (EWMA with different decay factors)
- Date/time components (year, month, day, weekday patterns)
- Volatility measures (rolling volatility, volatility ratios)
- Returns-based features (log returns, percentage returns)

**Objectives:**
- Capture temporal dependencies in market```ta
- Extract seasonality and cyclical patterns
- Create features for time-series modeling
- Enable prediction of future price movements```Support multiple forecasting horizons```hese features are essential for LSTM```RIMA, and other time-series prediction models.


In [1]:
# ============================================
# Time-Series Features - Library Imports and Setup
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
sns.set_theme(style='darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', '{:.6f}'.format)

print("✅ Time-series analysis libraries loaded successfully")
print("📊 Ready to extract time-series features")


✅ Time-series analysis libraries loaded successfully
📊 Ready to extract time-series features


In [2]:
# ============================================
# Load Market Data for Time-Series Analysis
# ============================================

# Load historical OHLCV data
data_path = '../data/raw/market_data.csv'

try:
    # Load real market data
    df = pd.read_csv(data_path, parse_dates=['date'], index_col='date')
    print(f"✅ Market data loaded from {data_path}")
    print(f"📊 Data shape: {df.shape}")
    
except FileNotFoundError:
    print("📝 Market data file not found. Generating realistic sample data...")
    
    # Generate realistic time-series data for demonstration
    np.random.seed(42)
    n_periods = 1000
    start_date = '2020-01-01'
    dates = pd.date_range(start=start_date, periods=n_periods, freq='D')
    
    # Generate realistic price movements with time dependencies
    initial_price = 100
    trend = 0.0002  # Small upward trend
    volatility = 0.02
    
    # Create correlated returns with memory
    returns = []
    prev_return = 0
    
    for i in range(n_periods):
        # Add trend, volatility, and memory effect
        random_shock = np.random.normal(0, volatility)
        memory_effect = 0.1 * prev_return  # Previous day influence
        daily_return = trend + memory_effect + random_shock
        returns.append(daily_return)
        prev_return = daily_return
    
    # Convert to prices
    prices = [initial_price]
    for ret in returns[1:]:
        prices.append(prices[-1] * (1 + ret))
    
    # Create OHLCV DataFrame
    df = pd.DataFrame(index=dates)
    df['close'] = prices
    df['open'] = df['close'].shift(1) * (1 + np.random.normal(0, 0.003, n_periods))
    df['high'] = np.maximum(df['open'], df['close']) * (1 + np.abs(np.random.normal(0, 0.008, n_periods)))
    df['low'] = np.minimum(df['open'], df['close']) * (1 - np.abs(np.random.normal(0, 0.008, n_periods)))
    df['volume'] = np.random.lognormal(15, 0.3, n_periods).astype(int)
    
    # Ensure OHLC relationships
    df['high'] = np.maximum(df['high'], df[['open', 'close']].max(axis=1))
    df['low'] = np.minimum(df['low'], df[['open', 'close']].min(axis=1))
    
    # Remove NaN values
    df = df.dropna()
    
    print("✅ Realistic sample time-series data generated")

# Display basic information
print(f"\n📈 Time-Series Dataset Overview:")
print(f"   • Date Range: {df.index.min().date()} to {df.index.max().date()}")
print(f"   • Total Records: {len(df):,}")
print(f"   • Frequency: Daily")
print(f"   • Columns: {list(df.columns)}")

print(f"\n📋 Data Preview:")
display(df.head())

print(f"\n📊 Basic Statistics:")
display(df.describe())


📝 Market data file not found. Generating realistic sample data...
✅ Realistic sample time-series data generated

📈 Time-Series Dataset Overview:
   • Date Range: 2020-01-02 to 2022-09-26
   • Total Records: 999
   • Frequency: Daily
   • Columns: ['close', 'open', 'high', 'low', 'volume']

📋 Data Preview:


,close,open,high,low,volume
2020-01-02,99.844814,100.277390,100.393326,99.157574,3238558
2020-01-03,101.142656,99.862676,101.783835,99.532246,3286734
2020-01-04,104.375221,100.946357,104.632369,99.421915,3766998
2020-01-05,104.240888,104.593853,106.178336,103.776763,2169359
2020-01-06,103.760187,104.363939,104.542021,102.651628,3905007



📊 Basic Statistics:


,close,open,high,low,volume
count,999.000000,999.000000,999.000000,999.000000,999.000000
mean,102.307410,102.274324,103.758731,100.797867,3365294.277277
std,24.851933,24.809056,25.190638,24.437352,1001563.418915
min,69.959378,69.575168,71.378241,69.258193,1260461.000000
25%,84.836253,84.729451,86.120471,83.588773,2665516.000000
50%,94.800443,94.846555,96.264000,93.445429,3252768.000000
75%,110.634067,110.641349,112.118710,109.202708,3962069.000000
max,187.949032,188.588305,190.686477,184.273699,8317506.000000


In [3]:
# ============================================
# Lag Features Creation
# ============================================

print("=" * 60)
print("LAG FEATURES CREATION")
print("=" * 60)

# Define lag periods for different time horizons
short_lags = [1, 2, 3]      # Short-term memory (1-3 days)
medium_lags = [5, 7, 10]    # Medium-term memory (1-2 weeks)
long_lags = [15, 20, 30]    # Long-term memory (3-4 weeks)

all_lags = short_lags + medium_lags + long_lags

# Create lag features for price variables
price_columns = ['close', 'open', 'high', 'low']
for col in price_columns:
    for lag in all_lags:
        df[f'{col}_lag_{lag}'] = df[col].shift(lag)

print(f"✅ Price lag features created for periods: {all_lags}")

# Create lag features for volume
for lag in all_lags:
    df[f'volume_lag_{lag}'] = df['volume'].shift(lag)

print(f"✅ Volume lag features created")

# Create lag features for returns (will calculate returns first)
df['daily_return'] = df['close'].pct_change()
df['log_return'] = np.log(df['close'] / df['close'].shift(1))

for lag in short_lags + medium_lags:  # Don't need too many return lags
    df[f'return_lag_{lag}'] = df['daily_return'].shift(lag)
    df[f'log_return_lag_{lag}'] = df['log_return'].shift(lag)

print(f"✅ Return lag features created")

# Price differences (momentum features)
for lag in [1, 5, 10]:
    df[f'price_diff_{lag}'] = df['close'] - df[f'close_lag_{lag}']
    df[f'price_diff_pct_{lag}'] = (df['close'] / df[f'close_lag_{lag}'] - 1) * 100

print(f"✅ Price difference features created")

# Display lag features summary
lag_columns = [col for col in df.columns if 'lag' in col or 'diff' in col]
print(f"\n📊 Lag Features Summary:")
print(f"   • Total lag features: {len(lag_columns)}")
print(f"   • Price lags: {len([c for c in lag_columns if any(p in c for p in price_columns)])}")
print(f"   • Volume lags: {len([c for c in lag_columns if 'volume_lag' in c])}")
print(f"   • Return lags: {len([c for c in lag_columns if 'return_lag' in c])}")
print(f"   • Difference features: {len([c for c in lag_columns if 'diff' in c])}")

print(f"\n📋 Lag Features Preview:")
display(df[['close'] + [f'close_lag_{lag}' for lag in [1, 5, 10]]].head(15))


LAG FEATURES CREATION
✅ Price lag features created for periods: [1, 2, 3, 5, 7, 10, 15, 20, 30]
✅ Volume lag features created
✅ Return lag features created
✅ Price difference features created

📊 Lag Features Summary:
   • Total lag features: 63
   • Price lags: 36
   • Volume lags: 9
   • Return lags: 12
   • Difference features: 6

📋 Lag Features Preview:


,close,close_lag_1,close_lag_5,close_lag_10
2020-01-02,99.844814,NaN,NaN,NaN
2020-01-03,101.142656,99.844814,NaN,NaN
2020-01-04,104.375221,101.142656,NaN,NaN
2020-01-05,104.240888,104.375221,NaN,NaN
2020-01-06,103.760187,104.240888,NaN,NaN
2020-01-07,107.010279,103.760187,99.844814,NaN
2020-01-08,109.009339,107.010279,101.142656,NaN
2020-01-09,108.211239,109.009339,104.375221,NaN
2020-01-10,109.327877,108.211239,104.240888,NaN
2020-01-11,108.449270,109.327877,103.760187,NaN


In [4]:
# ============================================
# Rolling Statistics Features
# ============================================

print("=" * 60)
print("ROLLING STATISTICS FEATURES")
print("=" * 60)

# Define rolling windows for different timeframes
short_windows = [3, 5, 7]       # Short-term patterns
medium_windows = [10, 15, 20]   # Medium-term patterns  
long_windows = [30, 50, 100]    # Long-term patterns

all_windows = short_windows + medium_windows + long_windows

# Rolling statistics for close price
for window in all_windows:
    # Basic statistics
    df[f'close_rollmean_{window}'] = df['close'].rolling(window=window).mean()
    df[f'close_rollstd_{window}'] = df['close'].rolling(window=window).std()
    df[f'close_rollmin_{window}'] = df['close'].rolling(window=window).min()
    df[f'close_rollmax_{window}'] = df['close'].rolling(window=window).max()
    
    # Position within rolling window
    df[f'close_rollposition_{window}'] = (df['close'] - df[f'close_rollmin_{window}']) / (df[f'close_rollmax_{window}'] - df[f'close_rollmin_{window}'])

print(f"✅ Rolling statistics created for windows: {all_windows}")

# Rolling statistics for volume
for window in short_windows + medium_windows:  # Volume doesn't need very long windows
    df[f'volume_rollmean_{window}'] = df['volume'].rolling(window=window).mean()
    df[f'volume_rollstd_{window}'] = df['volume'].rolling(window=window).std()
    df[f'volume_ratio_{window}'] = df['volume'] / df[f'volume_rollmean_{window}']

print(f"✅ Volume rolling statistics created")

# Rolling statistics for returns
for window in short_windows + medium_windows:
    df[f'return_rollmean_{window}'] = df['daily_return'].rolling(window=window).mean()
    df[f'return_rollstd_{window}'] = df['daily_return'].rolling(window=window).std()
    
    # Sharpe-like ratio (return/volatility)
    df[f'return_sharpe_{window}'] = df[f'return_rollmean_{window}'] / df[f'return_rollstd_{window}']

print(f"✅ Return rolling statistics created")

# Rolling correlations between price and volume
for window in medium_windows:
    df[f'price_volume_corr_{window}'] = df['close'].rolling(window=window).corr(df['volume'])

print(f"✅ Rolling correlations created")

# Rolling quantiles
for window in [10, 20, 50]:
    for quantile in [0.25, 0.75]:
        df[f'close_rollq{int(quantile*100)}_{window}'] = df['close'].rolling(window=window).quantile(quantile)

print(f"✅ Rolling quantiles created")

# Display rolling features summary
rolling_columns = [col for col in df.columns if 'roll' in col]
print(f"\n📊 Rolling Features Summary:")
print(f"   • Total rolling features: {len(rolling_columns)}")
print(f"   • Price rolling features: {len([c for c in rolling_columns if 'close_roll' in c])}")
print(f"   • Volume rolling features: {len([c for c in rolling_columns if 'volume_roll' in c])}")
print(f"   • Return rolling features: {len([c for c in rolling_columns if 'return_roll' in c])}")

print(f"\n📋 Rolling Statistics Preview:")
sample_rolling = ['close_rollmean_20', 'close_rollstd_20', 'close_rollposition_20', 'volume_ratio_20']
display(df[['close'] + sample_rolling].head(25))


ROLLING STATISTICS FEATURES
✅ Rolling statistics created for windows: [3, 5, 7, 10, 15, 20, 30, 50, 100]
✅ Volume rolling statistics created
✅ Return rolling statistics created
✅ Rolling correlations created
✅ Rolling quantiles created

📊 Rolling Features Summary:
   • Total rolling features: 75
   • Price rolling features: 51
   • Volume rolling features: 12
   • Return rolling features: 12

📋 Rolling Statistics Preview:


,close,close_rollmean_20,close_rollstd_20,close_rollposition_20,volume_ratio_20
2020-01-02,99.844814,NaN,NaN,NaN,NaN
2020-01-03,101.142656,NaN,NaN,NaN,NaN
2020-01-04,104.375221,NaN,NaN,NaN,NaN
2020-01-05,104.240888,NaN,NaN,NaN,NaN
2020-01-06,103.760187,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-01-22,94.379826,102.065954,5.763959,0.132664,0.681558
2020-01-23,94.510480,101.734345,6.005588,0.140245,0.613230
2020-01-24,91.849392,101.108054,6.358451,0.000000,1.343947
2020-01-25,90.609122,100.426466,6.725011,0.000000,0.859753


In [5]:
# ============================================
# Exponentially Weighted Features
# ============================================

print("=" * 60)
print("EXPONENTIALLY WEIGHTED FEATURES")
print("=" * 60)

# Define different alpha values for different decay rates
fast_alpha = [0.5, 0.7]      # Fast decay (recent data heavily weighted)
medium_alpha = [0.3, 0.4]    # Medium decay 
slow_alpha = [0.1, 0.2]      # Slow decay (longer memory)

all_alphas = fast_alpha + medium_alpha + slow_alpha

# Exponentially Weighted Moving Averages (EWMA) for prices
for alpha in all_alphas:
    df[f'close_ewm_alpha_{alpha}'] = df['close'].ewm(alpha=alpha, adjust=False).mean()
    df[f'close_ewm_std_{alpha}'] = df['close'].ewm(alpha=alpha, adjust=False).std()

print(f"✅ Price EWMA features created for alpha values: {all_alphas}")

# EWMA for volume
for alpha in fast_alpha + medium_alpha:  # Volume doesn't need slow decay
    df[f'volume_ewm_alpha_{alpha}'] = df['volume'].ewm(alpha=alpha, adjust=False).mean()
    df[f'volume_ewm_ratio_{alpha}'] = df['volume'] / df[f'volume_ewm_alpha_{alpha}']

print(f"✅ Volume EWMA features created")

# EWMA for returns
for alpha in all_alphas:
    df[f'return_ewm_mean_{alpha}'] = df['daily_return'].ewm(alpha=alpha, adjust=False).mean()
    df[f'return_ewm_std_{alpha}'] = df['daily_return'].ewm(alpha=alpha, adjust=False).std()

print(f"✅ Return EWMA features created")

# Span-based EWMA (alternative parameterization)
spans = [5, 10, 20, 50]
for span in spans:
    df[f'close_ewm_span_{span}'] = df['close'].ewm(span=span, adjust=False).mean()
    df[f'return_ewm_vol_span_{span}'] = df['daily_return'].ewm(span=span, adjust=False).std()

print(f"✅ Span-based EWMA features created for spans: {spans}")

# EWMA-based signals
df['ewma_signal_fast_slow'] = np.where(
    df['close_ewm_alpha_0.5'] > df['close_ewm_alpha_0.1'], 1, 0
)  # Fast EWMA above slow EWMA

df['ewma_price_position'] = (df['close'] - df['close_ewm_alpha_0.3']) / df['close_ewm_std_0.3']

print(f"✅ EWMA-based trading signals created")

# Display EWMA features summary
ewma_columns = [col for col in df.columns if 'ewm' in col]
print(f"\n📊 EWMA Features Summary:")
print(f"   • Total EWMA features: {len(ewma_columns)}")
print(f"   • Price EWMA: {len([c for c in ewma_columns if 'close_ewm' in c])}")
print(f"   • Volume EWMA: {len([c for c in ewma_columns if 'volume_ewm' in c])}")
print(f"   • Return EWMA: {len([c for c in ewma_columns if 'return_ewm' in c])}")

print(f"\n📋 EWMA Features Preview:")
sample_ewma = ['close_ewm_alpha_0.3', 'close_ewm_std_0.3', 'ewma_signal_fast_slow', 'ewma_price_position']
display(df[['close'] + sample_ewma].head(20))


EXPONENTIALLY WEIGHTED FEATURES
✅ Price EWMA features created for alpha values: [0.5, 0.7, 0.3, 0.4, 0.1, 0.2]
✅ Volume EWMA features created
✅ Return EWMA features created
✅ Span-based EWMA features created for spans: [5, 10, 20, 50]
✅ EWMA-based trading signals created

📊 EWMA Features Summary:
   • Total EWMA features: 42
   • Price EWMA: 16
   • Volume EWMA: 8
   • Return EWMA: 16

📋 EWMA Features Preview:


,close,close_ewm_alpha_0.3,close_ewm_std_0.3,ewma_signal_fast_slow,ewma_price_position
2020-01-02,99.844814,99.844814,NaN,0,NaN
2020-01-03,101.142656,100.234167,0.917712,1,0.989949
2020-01-04,104.375221,101.476483,2.479947,1,1.168871
2020-01-05,104.240888,102.305804,2.432317,1,0.795572
2020-01-06,103.760187,102.742119,2.109505,1,0.482610
2020-01-07,107.010279,104.022567,2.793060,1,1.069691
2020-01-08,109.009339,105.518599,3.436767,1,1.015705
2020-01-09,108.211239,106.326391,3.173429,1,0.593947
2020-01-10,109.327877,107.226837,3.054493,1,0.687853
2020-01-11,108.449270,107.593567,2.627078,1,0.325724


In [6]:
# ============================================
# Date/Time Component Features
# ============================================

print("=" * 60)
print("DATE/TIME COMPONENT FEATURES")
print("=" * 60)

# Basic date/time components
df['year'] = df.index.year
df['month'] = df.index.month
df['quarter'] = df.index.quarter
df['day'] = df.index.day
df['dayofweek'] = df.index.dayofweek  # 0=Monday, 6=Sunday
df['dayofyear'] = df.index.dayofyear
df['week_of_year'] = df.index.isocalendar().week

print("✅ Basic date/time components extracted")

# Binary indicators for special periods
df['is_monday'] = (df['dayofweek'] == 0).astype(int)
df['is_friday'] = (df['dayofweek'] == 4).astype(int)
df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
df['is_month_start'] = df.index.is_month_start.astype(int)
df['is_month_end'] = df.index.is_month_end.astype(int)
df['is_quarter_start'] = df.index.is_quarter_start.astype(int)
df['is_quarter_end'] = df.index.is_quarter_end.astype(int)

print("✅ Binary period indicators created")

# Cyclical encoding for periodic features (sine/cosine transformation)
# This preserves the cyclical nature of time features
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
df['dayofyear_sin'] = np.sin(2 * np.pi * df['dayofyear'] / 365.25)
df['dayofyear_cos'] = np.cos(2 * np.pi * df['dayofyear'] / 365.25)

print("✅ Cyclical encodings created")

# Time since epoch (for trend detection)
epoch_start = pd.Timestamp('2020-01-01')
df['days_since_epoch'] = (df.index - epoch_start).days
df['months_since_epoch'] = ((df.index.year - epoch_start.year) * 12 + 
                           (df.index.month - epoch_start.month))

print("✅ Temporal trend features created")

# Holiday/market effect indicators (US market examples)
def is_near_holiday(dates, holiday_months=[1, 7, 11, 12]):
    """Simple approximation for holiday effects"""
    return dates.month.isin(holiday_months).astype(int)

df['near_holiday'] = is_near_holiday(df.index)

# Market session indicators (trading day characteristics)
df['is_trading_day'] = (~df['is_weekend']).astype(int)  # Simplified - doesn't account for actual holidays

print("✅ Market calendar features created")

# Display date/time features summary
datetime_columns = [col for col in df.columns if any(x in col for x in 
                   ['year', 'month', 'quarter', 'day', 'week', 'is_', 'sin', 'cos', 'since', 'near', 'trading'])]
print(f"\n📊 Date/Time Features Summary:")
print(f"   • Total date/time features: {len(datetime_columns)}")
print(f"   • Basic components: {len([c for c in datetime_columns if c in ['year', 'month', 'quarter', 'day', 'dayofweek', 'dayofyear', 'week_of_year']])}")
print(f"   • Binary indicators: {len([c for c in datetime_columns if c.startswith('is_')])}")
print(f"   • Cyclical encodings: {len([c for c in datetime_columns if 'sin' in c or 'cos' in c])}")
print(f"   • Temporal trends: {len([c for c in datetime_columns if 'since' in c])}")

print(f"\n📋 Date/Time Features Preview:")
sample_datetime = ['year', 'month', 'dayofweek', 'is_weekend', 'month_sin', 'month_cos', 'is_month_end']
display(df[sample_datetime].head(10))

# Show unique values for categorical features
print(f"\n🔍 Categorical Feature Analysis:")
print(f"   • Unique years: {df['year'].unique()}")
print(f"   • Unique quarters: {df['quarter'].unique()}")
print(f"   • Weekend proportion: {df['is_weekend'].mean():.3f}")
print(f"   • Month-end proportion: {df['is_month_end'].mean():.3f}")


DATE/TIME COMPONENT FEATURES
✅ Basic date/time components extracted
✅ Binary period indicators created
✅ Cyclical encodings created
✅ Temporal trend features created
✅ Market calendar features created

📊 Date/Time Features Summary:
   • Total date/time features: 24
   • Basic components: 7
   • Binary indicators: 8
   • Cyclical encodings: 8
   • Temporal trends: 2

📋 Date/Time Features Preview:


,year,month,dayofweek,is_weekend,month_sin,month_cos,is_month_end
2020-01-02,2020,1,3,0,0.500000,0.866025,0
2020-01-03,2020,1,4,0,0.500000,0.866025,0
2020-01-04,2020,1,5,1,0.500000,0.866025,0
2020-01-05,2020,1,6,1,0.500000,0.866025,0
2020-01-06,2020,1,0,0,0.500000,0.866025,0
2020-01-07,2020,1,1,0,0.500000,0.866025,0
2020-01-08,2020,1,2,0,0.500000,0.866025,0
2020-01-09,2020,1,3,0,0.500000,0.866025,0
2020-01-10,2020,1,4,0,0.500000,0.866025,0
2020-01-11,2020,1,5,1,0.500000,0.866025,0



🔍 Categorical Feature Analysis:
   • Unique years: [2020 2021 2022]
   • Unique quarters: [1 2 3 4]
   • Weekend proportion: 0.286
   • Month-end proportion: 0.032


In [7]:
# ============================================
# Advanced Volatility and Risk Features
# ============================================

print("=" * 60)
print("VOLATILITY AND RISK FEATURES")
print("=" * 60)

# Realized volatility measures
volatility_windows = [5, 10, 20, 50]

for window in volatility_windows:
    # Standard deviation of returns
    df[f'volatility_{window}'] = df['daily_return'].rolling(window=window).std()
    
    # Annualized volatility
    df[f'volatility_annualized_{window}'] = df[f'volatility_{window}'] * np.sqrt(252)
    
    # Parkinson volatility (using high-low range)
    df[f'parkinson_vol_{window}'] = np.sqrt(
        (1 / (4 * np.log(2))) * 
        (np.log(df['high'] / df['low']) ** 2).rolling(window=window).mean()
    )

print(f"✅ Realized volatility measures created for windows: {volatility_windows}")

# Garman-Klass volatility (more efficient than close-to-close)
for window in [10, 20]:
    ln_hl = np.log(df['high'] / df['low'])
    ln_co = np.log(df['close'] / df['open'])
    
    df[f'garman_klass_vol_{window}'] = np.sqrt(
        (0.5 * ln_hl**2 - (2*np.log(2) - 1) * ln_co**2).rolling(window=window).mean()
    )

print(f"✅ Garman-Klass volatility created")

# Rogers-Satchell volatility
for window in [10, 20]:
    ln_ho = np.log(df['high'] / df['open'])
    ln_hc = np.log(df['high'] / df['close'])
    ln_lo = np.log(df['low'] / df['open'])
    ln_lc = np.log(df['low'] / df['close'])
    
    df[f'rogers_satchell_vol_{window}'] = np.sqrt(
        (ln_ho * ln_hc + ln_lo * ln_lc).rolling(window=window).mean()
    )

print(f"✅ Rogers-Satchell volatility created")

# Volatility ratios (regime detection)
df['vol_ratio_5_20'] = df['volatility_5'] / df['volatility_20']
df['vol_ratio_10_50'] = df['volatility_10'] / df['volatility_50']

# Volatility percentile (relative volatility position)
for window in [20, 50]:
    df[f'vol_percentile_{window}'] = df[f'volatility_{window}'].rolling(window=window*2).rank(pct=True)

print(f"✅ Volatility ratios and percentiles created")

# Return-based risk measures
for window in [20, 50]:
    # Value at Risk (VaR) approximation
    df[f'var_95_{window}'] = df['daily_return'].rolling(window=window).quantile(0.05)
    df[f'var_99_{window}'] = df['daily_return'].rolling(window=window).quantile(0.01)
    
    # Expected Shortfall (Conditional VaR)
    var_95 = df[f'var_95_{window}']
    df[f'cvar_95_{window}'] = df['daily_return'].rolling(window=window).apply(
        lambda x: x[x <= x.quantile(0.05)].mean() if len(x[x <= x.quantile(0.05)]) > 0 else x.quantile(0.05)
    )

print(f"✅ Risk measures (VaR, CVaR) created")

# Downside deviation
for window in [20, 50]:
    df[f'downside_deviation_{window}'] = df['daily_return'].rolling(window=window).apply(
        lambda x: np.sqrt(np.mean(np.minimum(x, 0)**2))
    )

# Maximum Drawdown
def calculate_rolling_max_drawdown(prices, window):
    """Calculate rolling maximum drawdown"""
    rolling_max = prices.rolling(window=window, min_periods=1).max()
    drawdown = (prices - rolling_max) / rolling_max
    return drawdown.rolling(window=window, min_periods=1).min()

for window in [20, 50]:
    df[f'max_drawdown_{window}'] = calculate_rolling_max_drawdown(df['close'], window)

print(f"✅ Downside risk measures created")

# Volatility clustering indicators
df['vol_regime_high'] = (df['volatility_20'] > df['volatility_20'].rolling(window=100).quantile(0.75)).astype(int)
df['vol_regime_low'] = (df['volatility_20'] < df['volatility_20'].rolling(window=100).quantile(0.25)).astype(int)

print(f"✅ Volatility regime indicators created")

# Display volatility features summary
volatility_columns = [col for col in df.columns if any(x in col for x in 
                     ['volatility', 'vol_', 'parkinson', 'garman', 'rogers', 'var_', 'cvar_', 'drawdown', 'downside'])]
print(f"\n📊 Volatility Features Summary:")
print(f"   • Total volatility features: {len(volatility_columns)}")
print(f"   • Standard volatility: {len([c for c in volatility_columns if c.startswith('volatility_')])}")
print(f"   • Advanced estimators: {len([c for c in volatility_columns if any(x in c for x in ['parkinson', 'garman', 'rogers'])])}")
print(f"   • Risk measures: {len([c for c in volatility_columns if any(x in c for x in ['var_', 'cvar_', 'downside', 'drawdown'])])}")
print(f"   • Volatility ratios: {len([c for c in volatility_columns if 'ratio' in c])}")

print(f"\n📋 Volatility Features Preview:")
sample_vol = ['volatility_20', 'vol_ratio_5_20', 'var_95_20', 'max_drawdown_20', 'vol_regime_high']
display(df[['daily_return'] + sample_vol].head(25))

# Volatility statistics
print(f"\n📊 Volatility Statistics:")
print(f"   • Average 20-day volatility: {df['volatility_20'].mean():.4f}")
print(f"   • High volatility regime periods: {df['vol_regime_high'].sum():,} ({df['vol_regime_high'].mean()*100:.1f}%)")
print(f"   • Low volatility regime periods: {df['vol_regime_low'].sum():,} ({df['vol_regime_low'].mean()*100:.1f}%)")


VOLATILITY AND RISK FEATURES
✅ Realized volatility measures created for windows: [5, 10, 20, 50]
✅ Garman-Klass volatility created
✅ Rogers-Satchell volatility created
✅ Volatility ratios and percentiles created
✅ Risk measures (VaR, CVaR) created
✅ Downside risk measures created
✅ Volatility regime indicators created

📊 Volatility Features Summary:
   • Total volatility features: 36
   • Standard volatility: 8
   • Advanced estimators: 8
   • Risk measures: 10
   • Volatility ratios: 2

📋 Volatility Features Preview:


,daily_return,volatility_20,vol_ratio_5_20,var_95_20,max_drawdown_20,vol_regime_high
2020-01-02,NaN,NaN,NaN,NaN,0.000000,0
2020-01-03,0.012999,NaN,NaN,NaN,0.000000,0
2020-01-04,0.031960,NaN,NaN,NaN,0.000000,0
2020-01-05,-0.001287,NaN,NaN,NaN,-0.001287,0
2020-01-06,-0.004611,NaN,NaN,NaN,-0.005893,0
...,...,...,...,...,...,...
2020-01-22,-0.001662,0.020800,1.035352,-0.037681,-0.157640,0
2020-01-23,0.001384,0.020501,1.039061,-0.037681,-0.157640,0
2020-01-24,-0.028157,0.019457,1.202143,-0.037681,-0.159872,0
2020-01-25,-0.013503,0.019487,1.037918,-0.037681,-0.171217,0



📊 Volatility Statistics:
   • Average 20-day volatility: 0.0193
   • High volatility regime periods: 260 (26.0%)
   • Low volatility regime periods: 219 (21.9%)


In [8]:
# ============================================
# Time-Series Features Summary and Export
# ============================================

print("=" * 60)
print("TIME-SERIES FEATURES SUMMARY AND EXPORT")
print("=" * 60)

# Identify all created features (excluding original OHLCV)
original_columns = ['open', 'high', 'low', 'close', 'volume']
time_series_features = [col for col in df.columns if col not in original_columns]

# Categorize features
feature_categories = {
    'lag_features': [col for col in time_series_features if 'lag' in col],
    'rolling_features': [col for col in time_series_features if 'roll' in col],
    'ewma_features': [col for col in time_series_features if 'ewm' in col],
    'datetime_features': [col for col in time_series_features if any(x in col for x in 
                         ['year', 'month', 'quarter', 'day', 'week', 'is_', 'sin', 'cos', 'since', 'near', 'trading'])],
    'volatility_features': [col for col in time_series_features if any(x in col for x in 
                           ['volatility', 'vol_', 'parkinson', 'garman', 'rogers', 'var_', 'cvar_', 'drawdown', 'downside'])],
    'return_features': [col for col in time_series_features if 'return' in col and 'lag' not in col and 'roll' not in col and 'ewm' not in col],
    'difference_features': [col for col in time_series_features if 'diff' in col],
    'signal_features': [col for col in time_series_features if 'signal' in col or 'regime' in col or 'position' in col]
}

print(f"📊 TIME-SERIES FEATURES SUMMARY:")
print(f"   • Total features created: {len(time_series_features)}")
for category, features in feature_categories.items():
    print(f"   • {category.replace('_', ' ').title()}: {len(features)}")

# Create clean dataset
ts_features_df = df[time_series_features].copy()

# Remove rows with excessive NaN values (due to rolling calculations and lags)
initial_rows = len(ts_features_df)
ts_features_df = ts_features_df.dropna()
final_rows = len(ts_features_df)

print(f"\n📈 Dataset After Cleaning:")
print(f"   • Initial rows: {initial_rows:,}")
print(f"   • Final rows: {final_rows:,}")
print(f"   • Rows removed: {initial_rows - final_rows:,}")
print(f"   • Data retention: {final_rows/initial_rows*100:.1f}%")
print(f"   • Date range: {ts_features_df.index.min().date()} to {ts_features_df.index.max().date()}")

# Feature preview
print(f"\n📋 Time-Series Features Preview:")
display(ts_features_df.head())

# Create outputs directory if it doesn't exist
import os
output_dir = './outputs'
os.makedirs(output_dir, exist_ok=True)

# Export time-series features
output_path = f'{output_dir}/time_series_features.csv'
ts_features_df.to_csv(output_path)
print(f"\n💾 Time-series features saved to: {output_path}")

# Export feature list by category
feature_list_path = f'{output_dir}/time_series_features_list.txt'
with open(feature_list_path, 'w') as f:
    f.write("Time-Series Features List\n")
    f.write("=" * 50 + "\n\n")
    f.write(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Total features: {len(time_series_features)}\n")
    f.write(f"Dataset shape: {ts_features_df.shape}\n\n")
    
    for category, features in feature_categories.items():
        f.write(f"\n{category.replace('_', ' ').upper()} ({len(features)} features):\n")
        f.write("-" * 40 + "\n")
        for i, feature in enumerate(features, 1):
            f.write(f"{i:3d}. {feature}\n")

print(f"📝 Feature list saved to: {feature_list_path}")

# Export feature correlation matrix for analysis
feature_corr = ts_features_df.corr()
corr_path = f'{output_dir}/time_series_features_correlation.csv'
feature_corr.to_csv(corr_path)
print(f"📊 Feature correlation matrix saved to: {corr_path}")

# Summary statistics
print(f"\n📊 Time-Series Features Summary Statistics:")
display(ts_features_df.describe())

# Feature importance hint (most variable features)
feature_variance = ts_features_df.var().sort_values(ascending=False)
print(f"\n🔝 Top 10 Most Variable Features:")
for i, (feature, variance) in enumerate(feature_variance.head(10).items(), 1):
    print(f"   {i:2d}. {feature}: {variance:.6f}")

print(f"\n✅ Time-series feature engineering completed successfully!")
print(f"   Ready for feature selection and model training")
print(f"   All outputs saved to: {output_dir}/")


TIME-SERIES FEATURES SUMMARY AND EXPORT
📊 TIME-SERIES FEATURES SUMMARY:
   • Total features created: 253
   • Lag Features: 57
   • Rolling Features: 75
   • Ewma Features: 42
   • Datetime Features: 24
   • Volatility Features: 36
   • Return Features: 8
   • Difference Features: 6
   • Signal Features: 13

📈 Dataset After Cleaning:
   • Initial rows: 999
   • Final rows: 850
   • Rows removed: 149
   • Data retention: 85.1%
   • Date range: 2020-05-30 to 2022-09-26

📋 Time-Series Features Preview:


,close_lag_1,close_lag_2,close_lag_3,close_lag_5,close_lag_7,close_lag_10,close_lag_15,close_lag_20,close_lag_30,open_lag_1,open_lag_2,open_lag_3,open_lag_5,open_lag_7,open_lag_10,open_lag_15,open_lag_20,open_lag_30,high_lag_1,high_lag_2,high_lag_3,high_lag_5,high_lag_7,high_lag_10,high_lag_15,high_lag_20,high_lag_30,low_lag_1,low_lag_2,low_lag_3,low_lag_5,low_lag_7,low_lag_10,low_lag_15,low_lag_20,low_lag_30,volume_lag_1,volume_lag_2,volume_lag_3,volume_lag_5,volume_lag_7,volume_lag_10,volume_lag_15,volume_lag_20,volume_lag_30,daily_return,log_return,return_lag_1,log_return_lag_1,return_lag_2,log_return_lag_2,return_lag_3,log_return_lag_3,return_lag_5,log_return_lag_5,return_lag_7,log_return_lag_7,return_lag_10,log_return_lag_10,price_diff_1,price_diff_pct_1,price_diff_5,price_diff_pct_5,price_diff_10,price_diff_pct_10,close_rollmean_3,close_rollstd_3,close_rollmin_3,close_rollmax_3,close_rollposition_3,close_rollmean_5,close_rollstd_5,close_rollmin_5,close_rollmax_5,close_rollposition_5,close_rollmean_7,close_rollstd_7,close_rollmin_7,close_rollmax_7,close_rollposition_7,close_rollmean_10,close_rollstd_10,close_rollmin_10,close_rollmax_10,close_rollposition_10,close_rollmean_15,close_rollstd_15,close_rollmin_15,close_rollmax_15,close_rollposition_15,close_rollmean_20,close_rollstd_20,close_rollmin_20,close_rollmax_20,close_rollposition_20,close_rollmean_30,close_rollstd_30,close_rollmin_30,close_rollmax_30,close_rollposition_30,close_rollmean_50,close_rollstd_50,close_rollmin_50,close_rollmax_50,close_rollposition_50,close_rollmean_100,close_rollstd_100,close_rollmin_100,close_rollmax_100,close_rollposition_100,volume_rollmean_3,volume_rollstd_3,volume_ratio_3,volume_rollmean_5,volume_rollstd_5,volume_ratio_5,volume_rollmean_7,volume_rollstd_7,volume_ratio_7,volume_rollmean_10,volume_rollstd_10,volume_ratio_10,volume_rollmean_15,volume_rollstd_15,volume_ratio_15,volume_rollmean_20,volume_rollstd_20,volume_ratio_20,return_rollmean_3,return_rollstd_3,return_sharpe_3,return_rollmean_5,return_rollstd_5,return_sharpe_5,return_rollmean_7,return_rollstd_7,return_sharpe_7,return_rollmean_10,return_rollstd_10,return_sharpe_10,return_rollmean_15,return_rollstd_15,return_sharpe_15,return_rollmean_20,return_rollstd_20,return_sharpe_20,price_volume_corr_10,price_volume_corr_15,price_volume_corr_20,close_rollq25_10,close_rollq75_10,close_rollq25_20,close_rollq75_20,close_rollq25_50,close_rollq75_50,close_ewm_alpha_0.5,close_ewm_std_0.5,close_ewm_alpha_0.7,close_ewm_std_0.7,close_ewm_alpha_0.3,close_ewm_std_0.3,close_ewm_alpha_0.4,close_ewm_std_0.4,close_ewm_alpha_0.1,close_ewm_std_0.1,close_ewm_alpha_0.2,close_ewm_std_0.2,volume_ewm_alpha_0.5,volume_ewm_ratio_0.5,volume_ewm_alpha_0.7,volume_ewm_ratio_0.7,volume_ewm_alpha_0.3,volume_ewm_ratio_0.3,volume_ewm_alpha_0.4,volume_ewm_ratio_0.4,return_ewm_mean_0.5,return_ewm_std_0.5,return_ewm_mean_0.7,return_ewm_std_0.7,return_ewm_mean_0.3,return_ewm_std_0.3,return_ewm_mean_0.4,return_ewm_std_0.4,return_ewm_mean_0.1,return_ewm_std_0.1,return_ewm_mean_0.2,return_ewm_std_0.2,close_ewm_span_5,return_ewm_vol_span_5,close_ewm_span_10,return_ewm_vol_span_10,close_ewm_span_20,return_ewm_vol_span_20,close_ewm_span_50,return_ewm_vol_span_50,ewma_signal_fast_slow,ewma_price_position,year,month,quarter,day,dayofweek,dayofyear,week_of_year,is_monday,is_friday,is_weekend,is_month_start,is_month_end,is_quarter_start,is_quarter_end,month_sin,month_cos,dayofweek_sin,dayofweek_cos,dayofyear_sin,dayofyear_cos,days_since_epoch,months_since_epoch,near_holiday,is_trading_day,volatility_5,volatility_annualized_5,parkinson_vol_5,volatility_10,volatility_annualized_10,parkinson_vol_10,volatility_20,volatility_annualized_20,parkinson_vol_20,volatility_50,volatility_annualized_50,parkinson_vol_50,garman_klass_vol_10,garman_klass_vol_20,rogers_satchell_vol_10,rogers_satchell_vol_20,vol_ratio_5_20,vol_ratio_10_50,vol_percentile_20,vol_percentile_50,var_95_20,var_99_20,cvar_95_20,var_95_50,var_99_50,cvar_95_50,downside_devi


💾 Time-series features saved to: ./outputs/time_series_features.csv
📝 Feature list saved to: ./outputs/time_series_features_list.txt
📊 Feature correlation matrix saved to: ./outputs/time_series_features_correlation.csv

📊 Time-Series Features Summary Statistics:


,close_lag_1,close_lag_2,close_lag_3,close_lag_5,close_lag_7,close_lag_10,close_lag_15,close_lag_20,close_lag_30,open_lag_1,open_lag_2,open_lag_3,open_lag_5,open_lag_7,open_lag_10,open_lag_15,open_lag_20,open_lag_30,high_lag_1,high_lag_2,high_lag_3,high_lag_5,high_lag_7,high_lag_10,high_lag_15,high_lag_20,high_lag_30,low_lag_1,low_lag_2,low_lag_3,low_lag_5,low_lag_7,low_lag_10,low_lag_15,low_lag_20,low_lag_30,volume_lag_1,volume_lag_2,volume_lag_3,volume_lag_5,volume_lag_7,volume_lag_10,volume_lag_15,volume_lag_20,volume_lag_30,daily_return,log_return,return_lag_1,log_return_lag_1,return_lag_2,log_return_lag_2,return_lag_3,log_return_lag_3,return_lag_5,log_return_lag_5,return_lag_7,log_return_lag_7,return_lag_10,log_return_lag_10,price_diff_1,price_diff_pct_1,price_diff_5,price_diff_pct_5,price_diff_10,price_diff_pct_10,close_rollmean_3,close_rollstd_3,close_rollmin_3,close_rollmax_3,close_rollposition_3,close_rollmean_5,close_rollstd_5,close_rollmin_5,close_rollmax_5,close_rollposition_5,close_rollmean_7,close_rollstd_7,close_rollmin_7,close_rollmax_7,close_rollposition_7,close_rollmean_10,close_rollstd_10,close_rollmin_10,close_rollmax_10,close_rollposition_10,close_rollmean_15,close_rollstd_15,close_rollmin_15,close_rollmax_15,close_rollposition_15,close_rollmean_20,close_rollstd_20,close_rollmin_20,close_rollmax_20,close_rollposition_20,close_rollmean_30,close_rollstd_30,close_rollmin_30,close_rollmax_30,close_rollposition_30,close_rollmean_50,close_rollstd_50,close_rollmin_50,close_rollmax_50,close_rollposition_50,close_rollmean_100,close_rollstd_100,close_rollmin_100,close_rollmax_100,close_rollposition_100,volume_rollmean_3,volume_rollstd_3,volume_ratio_3,volume_rollmean_5,volume_rollstd_5,volume_ratio_5,volume_rollmean_7,volume_rollstd_7,volume_ratio_7,volume_rollmean_10,volume_rollstd_10,volume_ratio_10,volume_rollmean_15,volume_rollstd_15,volume_ratio_15,volume_rollmean_20,volume_rollstd_20,volume_ratio_20,return_rollmean_3,return_rollstd_3,return_sharpe_3,return_rollmean_5,return_rollstd_5,return_sharpe_5,return_rollmean_7,return_rollstd_7,return_sharpe_7,return_rollmean_10,return_rollstd_10,return_sharpe_10,return_rollmean_15,return_rollstd_15,return_sharpe_15,return_rollmean_20,return_rollstd_20,return_sharpe_20,price_volume_corr_10,price_volume_corr_15,price_volume_corr_20,close_rollq25_10,close_rollq75_10,close_rollq25_20,close_rollq75_20,close_rollq25_50,close_rollq75_50,close_ewm_alpha_0.5,close_ewm_std_0.5,close_ewm_alpha_0.7,close_ewm_std_0.7,close_ewm_alpha_0.3,close_ewm_std_0.3,close_ewm_alpha_0.4,close_ewm_std_0.4,close_ewm_alpha_0.1,close_ewm_std_0.1,close_ewm_alpha_0.2,close_ewm_std_0.2,volume_ewm_alpha_0.5,volume_ewm_ratio_0.5,volume_ewm_alpha_0.7,volume_ewm_ratio_0.7,volume_ewm_alpha_0.3,volume_ewm_ratio_0.3,volume_ewm_alpha_0.4,volume_ewm_ratio_0.4,return_ewm_mean_0.5,return_ewm_std_0.5,return_ewm_mean_0.7,return_ewm_std_0.7,return_ewm_mean_0.3,return_ewm_std_0.3,return_ewm_mean_0.4,return_ewm_std_0.4,return_ewm_mean_0.1,return_ewm_std_0.1,return_ewm_mean_0.2,return_ewm_std_0.2,close_ewm_span_5,return_ewm_vol_span_5,close_ewm_span_10,return_ewm_vol_span_10,close_ewm_span_20,return_ewm_vol_span_20,close_ewm_span_50,return_ewm_vol_span_50,ewma_signal_fast_slow,ewma_price_position,year,month,quarter,day,dayofweek,dayofyear,week_of_year,is_monday,is_friday,is_weekend,is_month_start,is_month_end,is_quarter_start,is_quarter_end,month_sin,month_cos,dayofweek_sin,dayofweek_cos,dayofyear_sin,dayofyear_cos,days_since_epoch,months_since_epoch,near_holiday,is_trading_day,volatility_5,volatility_annualized_5,parkinson_vol_5,volatility_10,volatility_annualized_10,parkinson_vol_10,volatility_20,volatility_annualized_20,parkinson_vol_20,volatility_50,volatility_annualized_50,parkinson_vol_50,garman_klass_vol_10,garman_klass_vol_20,rogers_satchell_vol_10,rogers_satchell_vol_20,vol_ratio_5_20,vol_ratio_10_50,vol_percentile_20,vol_percentile_50,var_95_20,var_99_20,cvar_95_20,var_95_50,var_99_50,cvar_95_50,downside_devi


🔝 Top 10 Most Variable Features:
    1. volume_lag_1: 1004515914674.438599
    2. volume_lag_2: 1004135345998.183716
    3. volume_lag_10: 981589187622.035034
    4. volume_lag_7: 980059636885.520630
    5. volume_lag_5: 978906270473.351318
    6. volume_lag_3: 975242055935.374146
    7. volume_lag_20: 966394792294.270874
    8. volume_lag_15: 965004691951.177734
    9. volume_lag_30: 963645337814.052612
   10. volume_ewm_alpha_0.7: 539441919430.347046

✅ Time-series feature engineering completed successfully!
   Ready for feature selection and model training
   All outputs saved to: ./outputs/


## Time-Series Features Summary

### 🎯 Feature Categories Created

**Lag Features (Temporal Dependencies):**
- Price lags (1, 2, 3, 5, 7, 10, 15, 20, 30 days)
- Volume lags for market activity patterns
- Return lags for momentum analysis
- Price differences for trend detection

**Rolling Statistics (Moving Windows):**
- Rolling means, standard deviations, min/max
- Rolling quantiles and position indicators
- Volume ratios and correlations
- Multi-timeframe analysis (3-100 days)

**Exponentially Weighted Features:**
- EWMA with various decay factors (0.1-0.7)
- EWMA-based volatility measures
- Adaptive trend following indicators
- Fast/slow EWMA crossover signals

**Date/Time Components:**
- Calendar features (year, month, quarter, day)
- Cyclical encodings (sine/cosine transformations)
- Market session indicators
- Holiday and seasonal effects

**Advanced Volatility Measures:**
- Parkinson, Garman-Klass, Rogers-Satchell estimators
- Value at Risk (VaR) and Conditional VaR
- Maximum drawdown and downside deviation
- Volatility regime detection

### 🚀 Integration with StockPredictionPro

**For LSTM Models:**
- Sequence-ready lag features for temporal modeling
- Multi-scale rolling statistics for pattern recognition
- Volatility regimes for adaptive learning

**For Traditional ML:**
- Rich feature set for ensemble models
- Date/time features for seasonality capture
- Risk measures for portfolio optimization

**For Trading Strategies:**
- EWMA crossover signals for entry/exit
- Volatility regimes for position sizing
- Support/resistance from rolling statistics

### 📊 Next Steps

1. **Feature Selection**: Apply selection techniques to identify most predictive features
2. **Model Training**: Use features for LSTM, Random Forest, and ensemble models
3. **Backtesting**: Validate trading strategies using these features
4. **Real-time Implementation**: Deploy for live prediction and trading

### 🎯 Key Benefits

- **Temporal Modeling**: Captures short, medium, and long-term dependencies
- **Volatility Adaptation**: Multiple volatility measures for different market conditions
- **Seasonality**: Comprehensive calendar and cyclical features
- **Risk Management**: Advanced risk measures for portfolio optimization
